In [151]:
import json
import re

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [212]:
url = "https://boardgamegeek.com/boardgameexpansion/132436/last-will-messenger-promo-card"
pattern = r'GEEK\.geekitemPreload\s*=\s*({.*?});'
response = requests.get(url)

In [213]:
soup = BeautifulSoup(response.content, 'html5lib')
scripts = soup.find_all('script')

In [214]:
df = None
for script in scripts:
    script_data = script.string
    if script_data:
        match_found = re.search(pattern, script_data, re.DOTALL)
        if match_found:
            data = match_found.group(1)
            data = json.loads(data)
            df = pd.DataFrame(data)
            break

In [215]:
df.head()

,item,media,videogalleries
itemdata,"[{'datatype': 'geekitem_fielddata', 'fieldname...",NaN,NaN
relatedlinktypes,"[boardgameaccessory, contains, containedin, re...",NaN,NaN
classification_types,"[boardgamesubdomain, boardgamecategory, boardg...",NaN,NaN
linkedforum_types,"[{'title': 'Board Game Forums', 'linkedforum_i...",NaN,NaN
subtypename,Board Game,NaN,NaN


In [216]:
df = df.drop(['media', 'videogalleries'], axis = 1).T

In [217]:
df

,itemdata,relatedlinktypes,classification_types,linkedforum_types,subtypename,hide_collection_fields,honor_subtype,honor_linktype,credit_subtypes,show_geekbuddy_analysis,...,imagepagehref,imageurl,topimageurl,imageSets,itemstate,promoted_ad,cardsets,files,videos,galleries
item,"[{'datatype': 'geekitem_fielddata', 'fieldname...","[boardgameaccessory, contains, containedin, re...","[boardgamesubdomain, boardgamecategory, boardg...","[{'title': 'Board Game Forums', 'linkedforum_i...",Board Game,[weight],boardgamehonor,boardgamehonor,"[{'datatype': 'geekitem_linkdata', 'other_obje...",True,...,/image/1454867/last-will-messenger-promo-card,https://cf.geekdo-images.com/d2IZJvK4febukQMpo...,https://cf.geekdo-images.com/R4oKGi7vRQbg8N2dl...,{'square100': {'src': 'https://cf.geekdo-image...,approved,None,"{'cardSets': [{'notes': '', 'addon': False, 'n...",NaN,NaN,NaN


In [233]:
df.columns

Index(['itemdata', 'relatedlinktypes', 'classification_types',
       'linkedforum_types', 'subtypename', 'hide_collection_fields',
       'honor_subtype', 'honor_linktype', 'credit_subtypes',
       'show_geekbuddy_analysis', 'rankinfo', 'polls', 'stats',
       'relatedcounts', 'itemid', 'objecttype', 'objectid', 'label', 'labelpl',
       'shortlabel', 'shortlabelpl', 'type', 'id', 'href', 'canonical_link',
       'subtype', 'subtypes', 'versioninfo', 'name', 'alternatename',
       'yearpublished', 'minplayers', 'maxplayers', 'minplaytime',
       'maxplaytime', 'minage', 'override_rankable', 'targetco_url',
       'walmart_id', 'instructional_videoid', 'summary_videoid',
       'playthrough_videoid', 'focus_videoid', 'howtoplay_videoid',
       'bggstore_product', 'short_description', 'links', 'linkcounts',
       'secondarynamescount', 'alternatenamescount', 'primaryname',
       'alternatenames', 'description', 'wiki', 'website', 'imageid', 'images',
       'imagepagehref', 'ima

In [234]:
rank = float(df.iloc[0]['rankinfo'][0]['rank'])
average = float(df.iloc[0]['stats']['average'])
avgweight = float(df.iloc[0]['stats']['avgweight'])
expands = -1
if df.iloc[0]['links']['expandsboardgame']:
    expands = df.iloc[0]['links']['expandsboardgame'][0]['objectid']
print(expands)

97842


In [235]:
input_dict = {
    'id': df['id'].astype('int'), 
    'name': df['name'], 
    'expands': [int(expands)],
    'yearpublished': df['yearpublished'].astype('int'), 
    'rank': [rank],
    'average': [average], 
    'avgweight': [avgweight], 
    'minplayers': df['minplayers'].astype('int'),
    'maxplayers': df['maxplayers'].astype('int'), 
    'minplaytime': df['minplaytime'].astype('int'), 
    'maxplaytime': df['maxplaytime'].astype('int'), 
    'minage': df['minage'].astype('int'), 
    'short_description': df['short_description'], 
    'canonical_link': df['canonical_link']
}

In [236]:
input_df = pd.DataFrame(input_dict)
input_df

,id,name,expands,yearpublished,rank,average,avgweight,minplayers,maxplayers,minplaytime,maxplaytime,minage,short_description,canonical_link
item,132436,Last Will: Messenger Promo Card,97842,2012,0.0,7.15344,2.2353,2,5,60,60,13,None,https://boardgamegeek.com/boardgame/132436/las...


In [237]:
input_df.dtypes

id                     int32
name                  object
expands                int64
yearpublished          int32
rank                 float64
average              float64
avgweight            float64
minplayers             int32
maxplayers             int32
minplaytime            int32
maxplaytime            int32
minage                 int32
short_description     object
canonical_link        object
dtype: object

In [178]:
def get_data(category, df):
    result = []
    for elem in df.iloc[0]['links'][category]:
        result.append(elem['name'])
    return result

In [179]:
designers = get_data('boardgamedesigner', df)
artists = get_data('boardgameartist', df)
categories = get_data('boardgamecategory', df)
mechanics = get_data('boardgamemechanic', df)
families = get_data('boardgamefamily', df)

In [180]:
print(designers, artists, categories, mechanics, families)

['Vladimír Suchý'] ['Tomáš Kučerovský'] ['Card Game', 'Economic', 'Humor'] ['Action Points', 'Hand Management', 'Victory Points as a Resource', 'Worker Placement'] ['Country: England', 'Digital Implementations: Tabletopia', 'Mechanism: Tableau Building', 'Misc: Lowest Score Wins']


In [181]:
game_df = pd.read_csv("collection.csv")
game_df.head()

,objectname,objectid,rating,numplays,weight,own,fortrade,want,wanttobuy,wanttoplay,...,acquisitiondate,acquiredfrom,quantity,privatecomment,invlocation,invdate,version_publishers,version_languages,version_yearpublished,version_nickname
0,"13 Days: The Cuban Missile Crisis, 1962",177590,0.0,NaN,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1775: Rebellion,128996,0.0,2.0,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1812: The Invasion of Canada,94246,0.0,1.0,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1911 Amundsen vs Scott,124847,0.0,26.0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1911 Amundsen vs Scott: Expansion 4 – Food Depots,150029,0.0,NaN,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Looping Games,English; Spanish,2013.0,English/Spanish edition


In [182]:
game_ids = pd.Series(game_df['objectid'])
game_ids

0      177590
1      128996
2       94246
3      124847
4      150029
        ...  
647      2243
648    113490
649    284333
650    280575
651     27588
Name: objectid, Length: 652, dtype: int64